<a href="https://colab.research.google.com/github/yunjeong-chang/2021_Artificial-Intelligence/blob/main/11%EC%A3%BC%EC%B0%A8/11%EC%A3%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B51_MNIST_LeNet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST_CNN_LeNet5

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
# parameters
learning_rate = 0.9
training_epochs = 50
batch_size = 256

In [3]:
# TorchVision의 MNIST 영상이 28x28로 가공되어 있어, Original 입력 사이즈 32x32로 맞추기 위해
transforms = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

# MNIST dataset
# transform이 mnist_train & mnist_test에 반영되는게 아니라 train_loader & test_loader에서 반영되는 듯
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms,
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms,
                         download=True)

# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class LeNet5(torch.nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__() 

    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
    self.x1 = torch.nn.Sigmoid()
    #self.x1 = torch.nn.Tanh()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
    self.x3 = torch.nn.Sigmoid()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2) 

    #self.c5 = torch.nn.Conv2d(16,120,kernel_size=5,stride=1,padding=0)
    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Sigmoid()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Sigmoid()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        
    out = out.view(out.size(0),-1) # batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [5]:
class LeNet5_(torch.nn.Module):
  def __init__(self):
    super(LeNet5_,self).__init__() 

    self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
    self.x1 = torch.nn.Tanh()
    self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

    self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
    self.x3 = torch.nn.Tanh()
    self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2) 

    self.fl = torch.nn.Faltten()

    self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
    self.x5 = torch.nn.Tanh()
    
    self.f6 = torch.nn.Linear(120,84,bias=True)
    self.x6 = torch.nn.Tanh()
    
    self.output = torch.nn.Linear(84,10,bias=True)


  def forward(self,x):
    out = self.c1(x)
    out = self.x1(out)
    out = self.s2(out)
    out = self.c3(out)
    out = self.x3(out)
    out = self.s4(out)        
    out = out.view(out.size(0),-1) # batch x 1 x 1 x 120  
    out = self.f5(out)
    out = self.x5(out)
    out = self.f6(out)    
    out = self.x6(out)
    out = self.output(out)  

    return out

In [6]:
model = LeNet5().to(device)

In [7]:
criterion = torch.nn.CrossEntropyLoss().to(device)   
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [8]:
#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 2.31481123
[Epoch:    2] cost = 2.30339456
[Epoch:    3] cost = 2.30240297
[Epoch:    4] cost = 2.30240607
[Epoch:    5] cost = 2.30237103
[Epoch:    6] cost = 2.30200744
[Epoch:    7] cost = 2.30197287
[Epoch:    8] cost = 2.30187297
[Epoch:    9] cost = 2.30174351
[Epoch:   10] cost = 2.30137014
[Epoch:   11] cost = 2.30020571
[Epoch:   12] cost = 2.26812148
[Epoch:   13] cost = 1.79248357
[Epoch:   14] cost = 0.792940855
[Epoch:   15] cost = 0.440582812
[Epoch:   16] cost = 0.290651768
[Epoch:   17] cost = 0.218678385
[Epoch:   18] cost = 0.175645381
[Epoch:   19] cost = 0.150330588
[Epoch:   20] cost = 0.128533274
[Epoch:   21] cost = 0.116181493
[Epoch:   22] cost = 0.103414178
[Epoch:   23] cost = 0.0936107412
[Epoch:   24] cost = 0.0846407935
[Epoch:   25] cost = 0.0781238154
[Epoch:   26] cost = 0.0737076998
[Epoch:   27] cost = 0.0680258572
[Epoch:   28] cost = 0.0637129471
[Epoch:   29] cost = 0.0601467676
[Epoch:   30

In [9]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
  model.eval()

  for X, Y in test_loader:
    Y = Y.to(device)
    X = X.to(device)


    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)



Accuracy: 0.9884815705128205


Accuracy: 0.8358998107910156